# Data Cleansing with Airbnb

We're going to start by doing some exploratory data analysis & cleansing. We will be using the SF Airbnb rental dataset from [Inside Airbnb](http://insideairbnb.com/get-the-data.html).

<img src="https://files.training.databricks.com/images/301/sf.jpg" style="height: 200px; margin: 10px; border: 1px solid #ddd; padding: 10px"/>

## ![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) In this lesson you:<br>
 - Impute missing values
 - Identify & remove outliers

### Setting the default database and user name  
##### Substitute "renato" by your name in the `username` variable.

In [0]:
## Put your name here
username = "renato"

dbutils.widgets.text("username", username)
spark.sql(f"CREATE DATABASE IF NOT EXISTS dsacademy_embedded_wave3_{username}")
spark.sql(f"USE dsacademy_embedded_wave3_{username}")
spark.conf.set("spark.sql.shuffle.partitions", 40)

spark.sql("SET spark.databricks.delta.formatCheck.enabled = false")
spark.sql("SET spark.databricks.delta.properties.defaults.autoOptimize.optimizeWrite = true")

Out[1]: DataFrame[key: string, value: string]

By default Spark on Databricks works with files on DBFS, until you're explicitly changing the schema.  
But if you want to read a file using **spark.read** function in databricks you can use the prefix **file:** followed by the complete path to the file.   
https://spark.apache.org/docs/2.1.0/api/python/pyspark.sql.html#pyspark.sql.DataFrameReader

In [0]:
import os

In [0]:
datapath = os.path.join(os.getcwd(), "data", "airbnb", "listings.csv.gz")
datapath = "file://" + datapath
print(datapath)

file:///Workspace/Repos/renato.rocha-souza@rbinternational.com/Embedded_Data_Scientist/Module_B/Day3/data/airbnb/listings.csv.gz


Let's load the Airbnb dataset in.

In [0]:
rawDF = spark.read.csv(datapath, header="true", inferSchema="true", multiLine="true", escape='"')
rawDF.limit(10).display()

id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms,bathrooms_text,bedrooms,beds,amenities,price,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
15883,https://www.airbnb.com/rooms/15883,20220911230927,2022-09-12T00:00:00.000+0000,city scrape,b&b near Old Danube river,"Four rooms, each one differently and individually designed, really charming with lots of details. The interior: a potpourri of many different styles, completed with precious things brought along from our trips.The spaceOld Danube river, a short walk to one of the small streets here. Everywhere small houses or gardens, a small idyllic island far from hectic city life. Here we settled down a few years ago, built a house for us and the kids and decided: let’s add 4 more rooms. And thus this small b&b was created, anything but standard and Mozart kitsch.Guest accessfree wifi, many books about Vienna, great tipps from the hosts...","small and personalFour rooms at this B&B, each one differently and individually designed, really charming with lots tasteful details. The interior – a potpourri of many different styles – has been enhanced with precious souvenirs from our trips.free parkingYou arrive by car? No worries! Parking is no problem at all! It is possible for free on the street right in front of our house. There are always enough spaces available. Arrive and unload peacefully, and begin your stay without any stress whatsoever! recreation nearbyThe Old Danube River’s beautiful recreational area begins at the end of our street. Kilometers of walks beside the water offer you fun and relaxation without cars and noise. It is a natural treasure in the middle of the city.",https://a0.muscache.com/pictures/18eff738-a737-428d-b653-99f2e79145b9.jpg,62142,https://www.airbnb.com/users/show/62142,Eva,2009-12-11T00:00:00.000+0000,"Vienna, Austria",Mein größtes Hobby: Reisen! Am liebsten mit meinen 3 Kindern um ihnen die weite Welt zu zeigen. Eindrücke von allen möglichen Orten finden so Einfluss im Design vom the rooms. Ich freue mich auf neue Begegnungen!,within a day,50%,33%,f,https://a0.muscache.com/im/pictures/user/2416670c-a785-417e-b345-f74b368678c9.jpg?aki_policy=profile_small,https://a0.muscache.com/im/pictures/user/2416670c-a785-417e-b345-f74b368678c9.jpg?aki_policy=profile_x_medium,Donaustadt,4,6,"['email', 'phone']",t,t,"Vienna, Austria",Donaustadt,null,48.24262,16.42767,Room in bed and breakfast,Hotel room,3,null,1 private bath,1,2,"[""Essentials"", ""Heating"", ""High chair"", ""Hangers"", ""Wifi"", ""Air conditioning"", ""Patio or balcony"", ""Paid parking garage off premises"", ""Luggage dropoff allowed"", ""Shampoo"", ""Long term stays allowed"", ""Pack \u2019n play/Travel crib"", ""Hair dryer"", ""Breakfast"", ""Bed linens"", ""TV"", ""Smoke alarm"", ""Hot water""]",$110.00,1,365,1,1,365,365,1.0,365.0,nul

In [0]:
rawDF.columns

Out[6]: ['id',
 'listing_url',
 'scrape_id',
 'last_scraped',
 'source',
 'name',
 'description',
 'neighborhood_overview',
 'picture_url',
 'host_id',
 'host_url',
 'host_name',
 'host_since',
 'host_location',
 'host_about',
 'host_response_time',
 'host_response_rate',
 'host_acceptance_rate',
 'host_is_superhost',
 'host_thumbnail_url',
 'host_picture_url',
 'host_neighbourhood',
 'host_listings_count',
 'host_total_listings_count',
 'host_verifications',
 'host_has_profile_pic',
 'host_identity_verified',
 'neighbourhood',
 'neighbourhood_cleansed',
 'neighbourhood_group_cleansed',
 'latitude',
 'longitude',
 'property_type',
 'room_type',
 'accommodates',
 'bathrooms',
 'bathrooms_text',
 'bedrooms',
 'beds',
 'amenities',
 'price',
 'minimum_nights',
 'maximum_nights',
 'minimum_minimum_nights',
 'maximum_minimum_nights',
 'minimum_maximum_nights',
 'maximum_maximum_nights',
 'minimum_nights_avg_ntm',
 'maximum_nights_avg_ntm',
 'calendar_updated',
 'has_availability',
 'availab

For the sake of simplicity, only keep certain columns from this dataset. We will talk about feature selection later.

In [0]:
columnsToKeep = [
  "host_is_superhost",
  #"cancellation_policy",
  "instant_bookable",
  "host_total_listings_count",
  "neighbourhood_cleansed",
  "latitude",
  "longitude",
  "property_type",
  "room_type",
  "accommodates",
  #"bathrooms",
  "bedrooms",
  "beds",
  #"bed_type",
  "minimum_nights",
  "number_of_reviews",
  "review_scores_rating",
  "review_scores_accuracy",
  "review_scores_cleanliness",
  "review_scores_checkin",
  "review_scores_communication",
  "review_scores_location",
  "review_scores_value",
  "price"]

baseDF = rawDF.select(columnsToKeep)
baseDF.cache().count()
baseDF.limit(10).display()

host_is_superhost,instant_bookable,host_total_listings_count,neighbourhood_cleansed,latitude,longitude,property_type,room_type,accommodates,bedrooms,beds,minimum_nights,number_of_reviews,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,price
f,f,6,Donaustadt,48.24262,16.42767,Room in bed and breakfast,Hotel room,3,1,2,1,14,4.71,4.86,4.93,4.93,4.86,4.71,4.5,$110.00
t,t,3,Leopoldstadt,48.21924,16.37831,Entire rental unit,Entire home/apt,5,1,3,5,350,4.75,4.8,4.65,4.91,4.93,4.75,4.69,$69.00
t,t,19,Rudolfsheim-Fnfhaus,48.18434,16.32701,Entire rental unit,Entire home/apt,6,2,4,1,181,4.83,4.9,4.88,4.89,4.93,4.59,4.7,$145.00
f,f,6,Innere Stadt,48.21496,16.37161,Entire rental unit,Entire home/apt,2,1,1,2,100,4.64,4.73,4.55,4.8,4.91,4.89,4.59,$100.00
t,f,3,Leopoldstadt,48.21778,16.37847,Entire rental unit,Entire home/apt,3,null,2,5,347,4.65,4.77,4.51,4.93,4.95,4.86,4.58,$68.00
f,f,6,Innere Stadt,48.21351,16.37282,Entire rental unit,Entire home/apt,2,1,1,3,52,4.63,4.67,4.35,4.69,4.75,4.88,4.56,$99.00
t,f,4,Leopoldstadt,48.2176,16.38018,Private room in rental unit,Private room,2,1,2,2,117,4.77,4.74,4.68,4.8,4.75,4.81,4.71,$50.00
f,f,6,Innere Stadt,48.21318,16.37486,Entire rental unit,Entire home/apt,4,2,1,3,69,4.58,4.8,4.76,4.83,4.92,4.85,4.73,$140.00
f,t,1,Ottakring,48.22207,16.31594,Entire rental unit,Entire home/apt,4,2,2,3,50,4.87,4.94,4.71,4.94,4.96,4.4,4.73,$77.00
t,f,2,Favoriten,48.17437,16.39339,Entire condo,Entire home/apt,4,1,2,5,178,4.77,4.87,4.67,4.88,4.87,3.98,4.66,$87.00


### Fixing Data Types

Take a look at the schema above. You'll notice that the `price` field got picked up as string.  
For our task, we need it to be a numeric (double type) field. 

Let's fix that.

In [0]:
from pyspark.sql.functions import col, translate

fixedPriceDF = baseDF.withColumn("price", translate(col("price"), "$,", "").cast("double"))

fixedPriceDF.limit(10).display()

host_is_superhost,instant_bookable,host_total_listings_count,neighbourhood_cleansed,latitude,longitude,property_type,room_type,accommodates,bedrooms,beds,minimum_nights,number_of_reviews,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,price
f,f,6,Donaustadt,48.24262,16.42767,Room in bed and breakfast,Hotel room,3,1,2,1,14,4.71,4.86,4.93,4.93,4.86,4.71,4.5,110.0
t,t,3,Leopoldstadt,48.21924,16.37831,Entire rental unit,Entire home/apt,5,1,3,5,350,4.75,4.8,4.65,4.91,4.93,4.75,4.69,69.0
t,t,19,Rudolfsheim-Fnfhaus,48.18434,16.32701,Entire rental unit,Entire home/apt,6,2,4,1,181,4.83,4.9,4.88,4.89,4.93,4.59,4.7,145.0
f,f,6,Innere Stadt,48.21496,16.37161,Entire rental unit,Entire home/apt,2,1,1,2,100,4.64,4.73,4.55,4.8,4.91,4.89,4.59,100.0
t,f,3,Leopoldstadt,48.21778,16.37847,Entire rental unit,Entire home/apt,3,null,2,5,347,4.65,4.77,4.51,4.93,4.95,4.86,4.58,68.0
f,f,6,Innere Stadt,48.21351,16.37282,Entire rental unit,Entire home/apt,2,1,1,3,52,4.63,4.67,4.35,4.69,4.75,4.88,4.56,99.0
t,f,4,Leopoldstadt,48.2176,16.38018,Private room in rental unit,Private room,2,1,2,2,117,4.77,4.74,4.68,4.8,4.75,4.81,4.71,50.0
f,f,6,Innere Stadt,48.21318,16.37486,Entire rental unit,Entire home/apt,4,2,1,3,69,4.58,4.8,4.76,4.83,4.92,4.85,4.73,140.0
f,t,1,Ottakring,48.22207,16.31594,Entire rental unit,Entire home/apt,4,2,2,3,50,4.87,4.94,4.71,4.94,4.96,4.4,4.73,77.0
t,f,2,Favoriten,48.17437,16.39339,Entire condo,Entire home/apt,4,1,2,5,178,4.77,4.87,4.67,4.88,4.87,3.98,4.66,87.0


### Summary statistics

Two options:
* describe
* summary (describe + IQR)

**Question:** When to use IQR/median over mean? Vice versa?

In [0]:
display(fixedPriceDF.describe())

summary,host_is_superhost,instant_bookable,host_total_listings_count,neighbourhood_cleansed,latitude,longitude,property_type,room_type,accommodates,bedrooms,beds,minimum_nights,number_of_reviews,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,price
count,11795,11797,11795,11797,11797,11797,11797,11797,11797,10648,11649,11797,11797,9869,9774,9774,9774,9775,9773,9772,11797
mean,null,null,21.951589656634166,null,48.204832128843464,16.361390798461894,null,null,3.3154191743663644,1.3684259954921112,1.8950124474203793,6.818004577434941,35.61159616851742,4.67334279055627,4.7868262737875815,4.692784939635757,4.833127685696722,4.810543222506388,4.722745318735274,4.679234547687248,95.07883360176317
stddev,null,null,52.91740195195649,null,0.021399984080834645,0.0364909101528313,null,null,1.8156688982294793,0.8899037562441913,1.3422869374535251,28.42904205203276,66.26044025022497,0.6014365842466193,0.3870716872209593,0.4513291980541244,0.3583794383333711,0.39803037369055505,0.36720156267062043,0.4013966348394232,194.3411495925487
min,f,f,1,Alsergrund,48.10857,16.16986,Camper/RV,Entire home/apt,0,1,1,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
max,t,t,1339,Whring,48.32643,16.55566859090834,Tiny home,Shared room,16,19,35,1125,844,5.0,5.0,5.0,5.0,5.0,5.0,5.0,9270.0


In [0]:
display(fixedPriceDF.summary())

summary,host_is_superhost,instant_bookable,host_total_listings_count,neighbourhood_cleansed,latitude,longitude,property_type,room_type,accommodates,bedrooms,beds,minimum_nights,number_of_reviews,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,price
count,11795,11797,11795,11797,11797,11797,11797,11797,11797,10648,11649,11797,11797,9869,9774,9774,9774,9775,9773,9772,11797
mean,null,null,21.951589656634166,null,48.204832128843464,16.361390798461894,null,null,3.3154191743663644,1.3684259954921112,1.8950124474203793,6.818004577434941,35.61159616851742,4.67334279055627,4.7868262737875815,4.692784939635757,4.833127685696722,4.810543222506388,4.722745318735274,4.679234547687248,95.07883360176317
stddev,null,null,52.91740195195649,null,0.021399984080834645,0.0364909101528313,null,null,1.8156688982294793,0.8899037562441913,1.3422869374535251,28.42904205203276,66.26044025022497,0.6014365842466193,0.3870716872209593,0.4513291980541244,0.3583794383333711,0.39803037369055505,0.36720156267062043,0.4013966348394232,194.3411495925487
min,f,f,1,Alsergrund,48.10857,16.16986,Camper/RV,Entire home/apt,0,1,1,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25%,null,null,1,null,48.19107,16.33922,null,null,2,1,1,1,2,4.61,4.75,4.59,4.81,4.79,4.62,4.59,47.0
50%,null,null,3,null,48.20439,16.35991,null,null,3,1,2,2,9,4.83,4.89,4.83,4.93,4.93,4.81,4.76,71.0
75%,null,null,13,null,48.21854,16.38211,null,null,4,2,2,3,39,5.0,5.0,5.0,5.0,5.0,4.98,4.91,104.0
max,t,t,1339,Whring,48.32643,16.55566859090834,Tiny home,Shared room,16,19,35,1125,844,5.0,5.0,5.0,5.0,5.0,5.0,5.0,9270.0


### Getting rid of extreme values

Let's take a look at the *min* and *max* values of the `price` column:

In [0]:
display(fixedPriceDF.select("price").describe())

summary,price
count,11797
mean,95.07883360176317
stddev,194.3411495925487
min,0.0
max,9270.0


There are some super-expensive listings. But it's the data scientist's job to decide what to do with them. We can certainly filter the "free" Airbnbs though.

Let's see first how many listings we can find where the *price* is zero.

In [0]:
fixedPriceDF.filter(col("price") == 0).count()

Out[12]: 2

Now only keep rows with a strictly positive *price*.

In [0]:
posPricesDF = fixedPriceDF.filter(col("price") > 0)

Let's take a look at the *min* and *max* values of the *minimum_nights* column:

In [0]:
display(posPricesDF.select("minimum_nights").describe())

summary,minimum_nights
count,11795
mean,6.818991097922849
stddev,28.431351453339126
min,1
max,1125


In [0]:
display(posPricesDF
  .groupBy("minimum_nights").count()
  .orderBy(col("count").desc(), col("minimum_nights"))
)

minimum_nights,count
1,4004
2,3294
3,1788
4,542
30,446
5,418
7,308
14,158
6,125
28,123


A minimum stay of one year seems to be a reasonable limit here. Let's filter out those records where the *minimum_nights* is greater then 365:

In [0]:
minNightsDF = posPricesDF.filter(col("minimum_nights") <= 365)

display(minNightsDF)

host_is_superhost,instant_bookable,host_total_listings_count,neighbourhood_cleansed,latitude,longitude,property_type,room_type,accommodates,bedrooms,beds,minimum_nights,number_of_reviews,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,price
f,f,6,Donaustadt,48.24262,16.42767,Room in bed and breakfast,Hotel room,3,1,2,1,14,4.71,4.86,4.93,4.93,4.86,4.71,4.5,110.0
t,t,3,Leopoldstadt,48.21924,16.37831,Entire rental unit,Entire home/apt,5,1,3,5,350,4.75,4.8,4.65,4.91,4.93,4.75,4.69,69.0
t,t,19,Rudolfsheim-Fnfhaus,48.18434,16.32701,Entire rental unit,Entire home/apt,6,2,4,1,181,4.83,4.9,4.88,4.89,4.93,4.59,4.7,145.0
f,f,6,Innere Stadt,48.21496,16.37161,Entire rental unit,Entire home/apt,2,1,1,2,100,4.64,4.73,4.55,4.8,4.91,4.89,4.59,100.0
t,f,3,Leopoldstadt,48.21778,16.37847,Entire rental unit,Entire home/apt,3,null,2,5,347,4.65,4.77,4.51,4.93,4.95,4.86,4.58,68.0
f,f,6,Innere Stadt,48.21351,16.37282,Entire rental unit,Entire home/apt,2,1,1,3,52,4.63,4.67,4.35,4.69,4.75,4.88,4.56,99.0
t,f,4,Leopoldstadt,48.2176,16.38018,Private room in rental unit,Private room,2,1,2,2,117,4.77,4.74,4.68,4.8,4.75,4.81,4.71,50.0
f,f,6,Innere Stadt,48.21318,16.37486,Entire rental unit,Entire home/apt,4,2,1,3,69,4.58,4.8,4.76,4.83,4.92,4.85,4.73,140.0
f,t,1,Ottakring,48.22207,16.31594,Entire rental unit,Entire home/apt,4,2,2,3,50,4.87,4.94,4.71,4.94,4.96,4.4,4.73,77.0
t,f,2,Favoriten,48.17437,16.39339,Entire condo,Entire home/apt,4,1,2,5,178,4.77,4.87,4.67,4.88,4.87,3.98,4.66,87.0


### Nulls

There are a lot of different ways to handle null values. Sometimes, null can actually be a key indicator of the thing you are trying to predict (e.g. if you don't fill in certain portions of a form, probability of it getting approved decreases).

Some ways to handle nulls:
* Drop any records that contain nulls
* Numeric:
  * Replace them with mean/median/zero/etc.
* Categorical:
  * Replace them with the mode
  * Create a special category for null
* Use techniques like ALS which are designed to impute missing values
  
**If you do ANY imputation techniques for categorical/numerical features, you MUST include an additional field specifying that field was imputed.**

SparkML's Imputer (covered below) does not support imputation for categorical features.

### Impute: Cast to Double

SparkML's `Imputer` requires all fields be of type double [Python](https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.feature.Imputer)/[Scala](https://spark.apache.org/docs/latest/api/scala/#org.apache.spark.ml.feature.Imputer). Let's cast all integer fields to double.

In [0]:
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType

integerColumns = [x.name for x in minNightsDF.schema.fields if x.dataType == IntegerType()]
doublesDF = minNightsDF

for c in integerColumns:
  doublesDF = doublesDF.withColumn(c, col(c).cast("double"))

columns = "\n - ".join(integerColumns)
print(f"Columns converted from Integer to Double:\n - {columns}")

Columns converted from Integer to Double:
 - host_total_listings_count
 - accommodates
 - bedrooms
 - beds
 - minimum_nights
 - number_of_reviews


Add in dummy variable if we will impute any value.

In [0]:
from pyspark.sql.functions import when

imputeCols = [
  "bedrooms",
  #"bathrooms",
  "beds", 
  "review_scores_rating",
  "review_scores_accuracy",
  "review_scores_cleanliness",
  "review_scores_checkin",
  "review_scores_communication",
  "review_scores_location",
  "review_scores_value"
]

for c in imputeCols:
  doublesDF = doublesDF.withColumn(c + "_na", when(col(c).isNull(), 1.0).otherwise(0.0))

In [0]:
display(doublesDF.describe())

summary,host_is_superhost,instant_bookable,host_total_listings_count,neighbourhood_cleansed,latitude,longitude,property_type,room_type,accommodates,bedrooms,beds,minimum_nights,number_of_reviews,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,price,bedrooms_na,beds_na,review_scores_rating_na,review_scores_accuracy_na,review_scores_cleanliness_na,review_scores_checkin_na,review_scores_communication_na,review_scores_location_na,review_scores_value_na
count,11789,11789,11787,11789,11789,11789,11789,11789,11789,10644,11643,11789,11789,9862,9767,9767,9767,9768,9766,9765,11789,11789,11789,11789,11789,11789,11789,11789,11789,11789
mean,null,null,21.964197845083568,null,48.20483212494854,16.361392849621012,null,null,3.3160573415896173,1.3685644494550921,1.8948724555526926,6.377724997879379,35.601408092289425,4.673328939363195,4.786805569775756,4.6927224326814665,4.833116617180275,4.81047092547092,4.722740118779423,4.679256528417796,95.11866994656036,0.09712443803545678,0.012384426159979643,0.16345746034438885,0.17151581983204683,0.17151581983204683,0.17151581983204683,0.17143099499533462,0.17160064466875902,0.1716854695054712
stddev,null,null,52.93311232817838,null,0.021405675710426536,0.03650044280765981,null,null,1.8158889072156743,0.8900422995620187,1.3424639943607417,19.887887736108137,66.26414096165404,0.6016279347580344,0.38719771410906473,0.4514627182426419,0.35849263607229875,0.39814976421419007,0.36720004138951706,0.40148079582191265,194.39992348113,0.29613969778030147,0.11059877817445335,0.369798213705189,0.3769750626417025,0.3769750626417025,0.3769750626417025,0.3769011258891479,0.37704896581107056,0.37712283541699315
min,f,f,1.0,Alsergrund,48.10857,16.16986,Camper/RV,Entire home/apt,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
max,t,t,1339.0,Whring,48.32643,16.55566859090834,Tiny home,Shared room,16.0,19.0,35.0,365.0,844.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,9270.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [0]:
doublesDF.limit(10).display()

host_is_superhost,instant_bookable,host_total_listings_count,neighbourhood_cleansed,latitude,longitude,property_type,room_type,accommodates,bedrooms,beds,minimum_nights,number_of_reviews,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,price,bedrooms_na,beds_na,review_scores_rating_na,review_scores_accuracy_na,review_scores_cleanliness_na,review_scores_checkin_na,review_scores_communication_na,review_scores_location_na,review_scores_value_na
f,f,6.0,Donaustadt,48.24262,16.42767,Room in bed and breakfast,Hotel room,3.0,1.0,2.0,1.0,14.0,4.71,4.86,4.93,4.93,4.86,4.71,4.5,110.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
t,t,3.0,Leopoldstadt,48.21924,16.37831,Entire rental unit,Entire home/apt,5.0,1.0,3.0,5.0,350.0,4.75,4.8,4.65,4.91,4.93,4.75,4.69,69.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
t,t,19.0,Rudolfsheim-Fnfhaus,48.18434,16.32701,Entire rental unit,Entire home/apt,6.0,2.0,4.0,1.0,181.0,4.83,4.9,4.88,4.89,4.93,4.59,4.7,145.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
f,f,6.0,Innere Stadt,48.21496,16.37161,Entire rental unit,Entire home/apt,2.0,1.0,1.0,2.0,100.0,4.64,4.73,4.55,4.8,4.91,4.89,4.59,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
t,f,3.0,Leopoldstadt,48.21778,16.37847,Entire rental unit,Entire home/apt,3.0,null,2.0,5.0,347.0,4.65,4.77,4.51,4.93,4.95,4.86,4.58,68.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
f,f,6.0,Innere Stadt,48.21351,16.37282,Entire rental unit,Entire home/apt,2.0,1.0,1.0,3.0,52.0,4.63,4.67,4.35,4.69,4.75,4.88,4.56,99.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
t,f,4.0,Leopoldstadt,48.2176,16.38018,Private room in rental unit,Private room,2.0,1.0,2.0,2.0,117.0,4.77,4.74,4.68,4.8,4.75,4.81,4.71,50.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
f,f,6.0,Innere Stadt,48.21318,16.37486,Entire rental unit,Entire home/apt,4.0,2.0,1.0,3.0,69.0,4.58,4.8,4.76,4.83,4.92,4.85,4.73,140.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
f,t,1.0,Ottakring,48.22207,16.31594,Entire rental unit,Entire home/apt,4.0,2.0,2.0,3.0,50.0,4.87,4.94,4.71,4.94,4.96,4.4,4.73,77.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
t,f,2.0,Favoriten,48.17437,16.39339,Entire condo,Entire home/apt,4.0,1.0,2.0,5.0,178.0,4.77,4.87,4.67,4.88,4.87,3.98,4.66,87.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Transformers and Estimators

**Transformer**: Accepts a DataFrame as input, and returns a new DataFrame with one or more columns appended to it. Transformers do not learn any parameters from your
data and simply apply rule-based transformations to either prepare data for model training or generate predictions using a trained MLlib model. They have a `.transform()` method.

**Estimator**: Learns (or "fits") parameters from your DataFrame via a `.fit()` method and returns a Model, which is a transformer.

In [0]:
from pyspark.ml.feature import Imputer

imputer = Imputer(strategy="median", inputCols=imputeCols, outputCols=imputeCols)

imputerModel = imputer.fit(doublesDF)
imputedDF = imputerModel.transform(doublesDF)

In [0]:
imputedDF.limit(10).display()

host_is_superhost,instant_bookable,host_total_listings_count,neighbourhood_cleansed,latitude,longitude,property_type,room_type,accommodates,bedrooms,beds,minimum_nights,number_of_reviews,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,price,bedrooms_na,beds_na,review_scores_rating_na,review_scores_accuracy_na,review_scores_cleanliness_na,review_scores_checkin_na,review_scores_communication_na,review_scores_location_na,review_scores_value_na
f,f,6.0,Donaustadt,48.24262,16.42767,Room in bed and breakfast,Hotel room,3.0,1.0,2.0,1.0,14.0,4.71,4.86,4.93,4.93,4.86,4.71,4.5,110.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
t,t,3.0,Leopoldstadt,48.21924,16.37831,Entire rental unit,Entire home/apt,5.0,1.0,3.0,5.0,350.0,4.75,4.8,4.65,4.91,4.93,4.75,4.69,69.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
t,t,19.0,Rudolfsheim-Fnfhaus,48.18434,16.32701,Entire rental unit,Entire home/apt,6.0,2.0,4.0,1.0,181.0,4.83,4.9,4.88,4.89,4.93,4.59,4.7,145.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
f,f,6.0,Innere Stadt,48.21496,16.37161,Entire rental unit,Entire home/apt,2.0,1.0,1.0,2.0,100.0,4.64,4.73,4.55,4.8,4.91,4.89,4.59,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
t,f,3.0,Leopoldstadt,48.21778,16.37847,Entire rental unit,Entire home/apt,3.0,1.0,2.0,5.0,347.0,4.65,4.77,4.51,4.93,4.95,4.86,4.58,68.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
f,f,6.0,Innere Stadt,48.21351,16.37282,Entire rental unit,Entire home/apt,2.0,1.0,1.0,3.0,52.0,4.63,4.67,4.35,4.69,4.75,4.88,4.56,99.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
t,f,4.0,Leopoldstadt,48.2176,16.38018,Private room in rental unit,Private room,2.0,1.0,2.0,2.0,117.0,4.77,4.74,4.68,4.8,4.75,4.81,4.71,50.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
f,f,6.0,Innere Stadt,48.21318,16.37486,Entire rental unit,Entire home/apt,4.0,2.0,1.0,3.0,69.0,4.58,4.8,4.76,4.83,4.92,4.85,4.73,140.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
f,t,1.0,Ottakring,48.22207,16.31594,Entire rental unit,Entire home/apt,4.0,2.0,2.0,3.0,50.0,4.87,4.94,4.71,4.94,4.96,4.4,4.73,77.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
t,f,2.0,Favoriten,48.17437,16.39339,Entire condo,Entire home/apt,4.0,1.0,2.0,5.0,178.0,4.77,4.87,4.67,4.88,4.87,3.98,4.66,87.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


OK, our data is cleansed now. Let's save this DataFrame to a Database so that we can start building models with it.
Delta

In [0]:
deltaPath = os.path.join("/", "tmp", username)    #If we were writing to the root folder and not to the DBFS
if not os.path.exists(deltaPath):
    os.mkdir(deltaPath)
    
print(deltaPath)

/tmp/renato


In [0]:
# Converting Spark DataFrame to Delta Table
dbutils.fs.rm(deltaPath, True)
imputedDF.write.format("delta").mode("overwrite").save(deltaPath)

### We are going to use this database in the next notebooks!

Code modified and enhanced from 2020 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="http://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="http://help.databricks.com/">Support</a>